# Create dataset

In [1]:
import PyPDF2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def extract_document_outline(pdf_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        # Create a PdfFileReader object
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Get the number of pages in the PDF
        num_pages = len(pdf_reader.pages)
        
        # Get the document outline
        document_outline = pdf_reader.outline
        
        # Initialize lists to store levels, titles, page numbers, and parent titles
        levels = []
        titles = []
        page_numbers = []
        parent_titles = []
        
        # Define a recursive function to traverse the document outline
        def traverse_outline(outline_items, level=0, parent_title=None):
            for item in outline_items:
                if isinstance(item, list):
                    # Handle nested outlines
                    traverse_outline(item, level + 1, parent_title)
                elif isinstance(item, dict):
                    # Extract title and page number
                    title = item.get('/Title')
                    page_number = item.get('/Page')
                    
                    if title and page_number:
                        # Append to the lists
                        levels.append(level)
                        titles.append(title)
                        page_numbers.append(pdf_reader.get_page_number(page_number) + 1)  # Adjust index
                        parent_titles.append(parent_title)
                    
                    # Update parent title for child items
                    parent_title = title
                    
        # Start traversing the document outline
        traverse_outline(document_outline)
        
        # Create a DataFrame from the extracted data
        df = pd.DataFrame({
            'Level': levels,
            'Title': titles,
            'Page Number': page_numbers,
            'Parent Title': parent_titles
        })
        
        return df

In [3]:
def extract_text_from_page(pdf_path, page_numbers):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        # Create a PdfFileReader object
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Initialize list to store text
        all_text = []
        
        # Extract text from specified page range
        for page_num in page_numbers:
            page = pdf_reader.pages[page_num - 1]  # Adjust index
            text = page.extract_text()
            all_text.append(text)
        
        return all_text

In [4]:
def add_description(df, df2, pdf_path):
    # Extract page numbers from the DataFrame
    page_numbers = df['Page Number'].tolist()
    
    # Extract text from each page
    all_text = extract_text_from_page(pdf_path, page_numbers)
    train_text = []
    test_text = []
    for i in range(len(all_text)):
        total_length = len(all_text[i])
        train_text_entry = all_text[i][:int(0.8*total_length)]
        test_text_entry = all_text[i][int(0.8*total_length):]
        train_text.append(train_text_entry)
        test_text.append(test_text_entry)
    # Add description column to the DataFrame
    df['Description'] = train_text
    df2['Description'] = test_text
    
    return df, df2

In [5]:
os_book_path = './../dataset/pdf/OS_Main book.pdf'

In [6]:
document_outline_df = extract_document_outline(os_book_path)
document_outline_df_2 = document_outline_df.copy()

In [79]:
document_outline_df_2

,Level,Title,Page Number,Parent Title
0,0,Cover,1,None
1,0,Title Page,5,Cover
2,0,Copyright,6,Title Page
3,0,Preface,9,Copyright
4,0,Contents,23,Preface
...,...,...,...,...
905,2,Bibliography,1191,Further Reading
906,0,Credits,1193,PART TEN APPENDICES
907,0,Index,1195,Credits
908,0,Glossary,1237,Index


In [80]:
document_outline_df

,Level,Title,Page Number,Parent Title
0,0,Cover,1,None
1,0,Title Page,5,Cover
2,0,Copyright,6,Title Page
3,0,Preface,9,Copyright
4,0,Contents,23,Preface
...,...,...,...,...
905,2,Bibliography,1191,Further Reading
906,0,Credits,1193,PART TEN APPENDICES
907,0,Index,1195,Credits
908,0,Glossary,1237,Index


In [8]:
document_outline_df, document_outline_df_2 = add_description(document_outline_df, document_outline_df_2, os_book_path)

In [9]:
document_outline_df

,Level,Title,Page Number,Parent Title,Description
0,0,Cover,1,None,
1,0,Title Page,5,Cover,OPERATING\nSYSTEM\nCONCEPTS\nABRAHAM SILBERSCH...
2,0,Copyright,6,Title Page,Publisher Laurie Rosatone \nEditorial Direc...
3,0,Preface,9,Copyright,Preface\nOperating systems are an essential pa...
4,0,Contents,23,Preface,Contents\nPART ONE\n OVERVIEW\nChapter 1 Intro...
...,...,...,...,...,...
905,2,Bibliography,1191,Further Reading,Further Reading 25\nMach uses lightweight proc...
906,0,Credits,1193,PART TEN APPENDICES,Credits\n•Figure 1.14: From Hennesy and Patter...
907,0,Index,1195,Credits,"Index\n4-byte pages, 363, 364\n32-byte memory,..."
908,0,Glossary,1237,Index,G-150-percent rule A statistical fi nding th...


In [10]:
document_outline_df_2

,Level,Title,Page Number,Parent Title,Description
0,0,Cover,1,None,
1,0,Title Page,5,Cover,"\n8FTUNJOTUFS$PMMFHF\n7(17+(',7,21\n"
2,0,Copyright,6,Title Page,\n 9781119320913 (enhanced ePub) \nSubjects...
3,0,Preface,9,Copyright,we present a large\nnumberofexamplesthatperta...
4,0,Contents,23,Preface,s 116\n3.4 Interprocess Communication 123\n3.5...
...,...,...,...,...,...
905,2,Bibliography,1191,Further Reading,"rtson, “Accent: A Com-\nmunication-Oriented Ne..."
906,0,Credits,1193,PART TEN APPENDICES,"Halsall, Data Communications, Computer\nNetwo..."
907,0,Index,1195,Credits,"aph directories, 547-549\nadditional-reference..."
908,0,Glossary,1237,Index,on I/O bus.\nadvisory fi le-lock mechanism A ...


In [84]:
document_outline_df.to_csv('document_outline.csv', index=False)

In [85]:
document_outline_df_2.to_csv('document_outline_2.csv', index=False)

## Data Preprocessing:

In [11]:
from pathlib import Path
import pandas as pd
train_path = Path("./document_outline.csv")
valid_path = Path("./document_outline_2.csv")
train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

In [12]:
import regex

def preprocess_df(input_df, X, Y):
    df = input_df.copy()
    
    df['Description'] = input_df[X].apply(lambda row: ' '.join(row.map(str)), axis=1)
    
    pattern = regex.compile(r'\W+', regex.UNICODE)
    df['Description'] = df['Description'].apply(lambda x: pattern.sub(' ', str(x)))

    df['cat1'] = ''
    df['cat2'] = ''
    df['cat3'] = ''
    
    for index, row in df.iterrows():
        if row['Level'] == 0:
            df.at[index, 'cat1'] = row['Title']
        elif row['Level'] == 1:
            parent_title = df.loc[df.index < index, 'Title'].iloc[-1] 
            df.at[index, 'cat1'] = parent_title
            df.at[index, 'cat2'] = row['Title']
        elif row['Level'] == 2:
            parent_title_level1 = df.loc[df.index < index, 'Title'].iloc[-2] 
            parent_title_level2 = df.loc[df.index < index, 'Title'].iloc[-1]  
            df.at[index, 'cat1'] = parent_title_level1
            df.at[index, 'cat2'] = parent_title_level2
            df.at[index, 'cat3'] = row['Title']
    
    return df[['Description', 'cat1', 'cat2', 'cat3']]


In [13]:
train_df.head()

,Level,Title,Page Number,Parent Title,Description
0,0,Cover,1,NaN,NaN
1,0,Title Page,5,Cover,OPERATING\nSYSTEM\nCONCEPTS\nABRAHAM SILBERSCH...
2,0,Copyright,6,Title Page,Publisher Laurie Rosatone \nEditorial Direc...
3,0,Preface,9,Copyright,Preface\nOperating systems are an essential pa...
4,0,Contents,23,Preface,Contents\nPART ONE\n OVERVIEW\nChapter 1 Intro...


In [14]:
X = ["Description"]
Y = ["Title", "Parent Title", "Level"]
preprocessed_train_df = preprocess_df(train_df, X, Y)
preprocessed_train_df.to_csv("preprocessed_train.csv")
preprocessed_valid_df = preprocess_df(valid_df, X, Y)
preprocessed_valid_df.to_csv("preprocessed_valid.csv")

In [23]:
train_df

,Level,Title,Page Number,Parent Title,Description
0,0,Cover,1,NaN,NaN
1,0,Title Page,5,Cover,OPERATING\nSYSTEM\nCONCEPTS\nABRAHAM SILBERSCH...
2,0,Copyright,6,Title Page,Publisher Laurie Rosatone \nEditorial Direc...
3,0,Preface,9,Copyright,Preface\nOperating systems are an essential pa...
4,0,Contents,23,Preface,Contents\nPART ONE\n OVERVIEW\nChapter 1 Intro...
...,...,...,...,...,...
905,2,Bibliography,1191,Further Reading,Further Reading 25\nMach uses lightweight proc...
906,0,Credits,1193,PART TEN APPENDICES,Credits\n•Figure 1.14: From Hennesy and Patter...
907,0,Index,1195,Credits,"Index\n4-byte pages, 363, 364\n32-byte memory,..."
908,0,Glossary,1237,Index,G-150-percent rule A statistical fi nding th...


In [24]:
valid_df

,Level,Title,Page Number,Parent Title,Description
0,0,Cover,1,NaN,NaN
1,0,Title Page,5,Cover,"\n8FTUNJOTUFS$PMMFHF\n7(17+(',7,21\n"
2,0,Copyright,6,Title Page,\n 9781119320913 (enhanced ePub) \nSubjects...
3,0,Preface,9,Copyright,we present a large\nnumberofexamplesthatperta...
4,0,Contents,23,Preface,s 116\n3.4 Interprocess Communication 123\n3.5...
...,...,...,...,...,...
905,2,Bibliography,1191,Further Reading,"rtson, “Accent: A Com-\nmunication-Oriented Ne..."
906,0,Credits,1193,PART TEN APPENDICES,"Halsall, Data Communications, Computer\nNetwo..."
907,0,Index,1195,Credits,"aph directories, 547-549\nadditional-reference..."
908,0,Glossary,1237,Index,on I/O bus.\nadvisory fi le-lock mechanism A ...


In [32]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

language_stop_words = stopwords.words('english')

vectorizer = TfidfVectorizer()
vectorizer.fit(preprocessed_train_df['Description'].apply(lambda x: np.str_(x)))
X_train = vectorizer.transform(preprocessed_train_df['Description'].apply(lambda x: np.str_(x)))

X_valid = vectorizer.transform(preprocessed_valid_df['Description'].apply(lambda x: np.str_(x)))


oe = OrdinalEncoder()
Y_columns = ['cat1', 'cat2', 'cat3']
oe.fit(preprocessed_train_df[Y_columns].values)
Y_train = oe.transform(preprocessed_train_df[Y_columns].values)
Y_valid = oe.transform(preprocessed_valid_df[Y_columns].values)

print('X training shape', X_train.shape, X_train.dtype)
print('Y training shape', Y_train.shape, Y_train.dtype)
print('X Valid shape', X_valid.shape, X_valid.dtype)
print('Y Valid shape', Y_train.shape, Y_valid.dtype)

X training shape (910, 26922) float64
Y training shape (910, 3) float64
X Valid shape (910, 26922) float64
Y Valid shape (910, 3) float64


## Analyzing dataset

In [91]:
import matplotlib.pyplot as plt
import seaborn as sns

### Heat Map:
**use**:  visualizing the correlation between the categories (cat1, cat2, cat3).

## Classification

In [17]:
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import SGDClassifier


clf=ClassifierChain(SGDClassifier(random_state=0, class_weight='balanced', n_jobs=-1))
clf.fit(X_train, Y_train)

ClassifierChain(base_estimator=SGDClassifier(class_weight='balanced', n_jobs=-1,
                                             random_state=0))

## Evaluation

In [34]:
from sklearn.metrics import jaccard_score, f1_score, make_scorer

def concat_categories_level2(Y):
    return np.apply_along_axis(lambda a: str(a[0]) + '-' + str(a[1]), 1, Y)

def concat_categories_level3(Y):
    return np.apply_along_axis(lambda a: str(a[0]) + '-' + str(a[1]+'-'+ str(a[2])), 1, Y)

def js_0(y, y_pred, **kwargs):
    return jaccard_score(y[:,0], y_pred[:,0], average='micro')

def js_1(y, y_pred, **kwargs):
    return jaccard_score(y[:,1], y_pred[:,1], average='micro')

def f1_0(y, y_pred, **kwargs):
    return f1_score(y[:,0], y_pred[:,0], average='micro')

def f1_1(y, y_pred, **kwargs):
    return f1_score(y[:,1], y_pred[:,1], average='micro')

# Score for predicting 'category_1-category_2' (concatenated strings)
def js_01(y, y_pred, **kwargs):
    return jaccard_score(concat_categories_level2(y), concat_categories_level2(y_pred), average='micro')

def f1_01(y, y_pred, **kwargs):
    return f1_score(concat_categories_level2(y), concat_categories_level2(y_pred), average='micro')

def js_001(y, y_pred, **kwargs):
    return jaccard_score(concat_categories_level3(y), concat_categories_level3(y_pred), average='micro')

def f1_001(y, y_pred, **kwargs):
    return f1_score(concat_categories_level3(y), concat_categories_level3(y_pred), average='micro')

js_0_scorer = make_scorer(score_func=js_0, greater_is_better=True)
js_1_scorer = make_scorer(score_func=js_1, greater_is_better=True)
js_01_scorer = make_scorer(score_func=js_01, greater_is_better=True)
js_001_scorer = make_scorer(score_func=js_001, greater_is_better=True)
f1_0_scorer = make_scorer(score_func=f1_0, greater_is_better=True)
f1_1_scorer = make_scorer(score_func=f1_1, greater_is_better=True)
f1_01_scorer = make_scorer(score_func=f1_01, greater_is_better=True)
f1_001_scorer = make_scorer(score_func=f1_001, greater_is_better=True)

In [ ]:
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Create the SGD classifier
sgd_clf = SGDClassifier(random_state=0, class_weight='balanced', n_jobs=-1)

# Initialize ClassifierChain with SGDClassifier as the base estimator
clf = ClassifierChain(base_estimator=sgd_clf)

# Define the parameter grid
param_grid = {
    'base_estimator__alpha': [0.0001, 0.001, 0.01, 0.1],
    'base_estimator__loss': ['hinge', 'log', 'modified_huber'],
    'base_estimator__penalty': ['l1', 'l2'],
    'base_estimator__max_iter': [1000, 2000, 3000]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring=f1_01_scorer)

# Fit the grid search to the data
grid_search.fit(X_train, Y_train)

# Print the best parameters found
print("Best parameters found:")
print(grid_search.best_params_)

# Get the best estimator
best_clf = grid_search.best_estimator_

# Predict on the validation set
Y_valid_pred = best_clf.predict(X_valid)

# Compute the F1 score
f1_micro = f1_01_scorer(Y_valid, Y_valid_pred, average='micro')
print("Validation F1 Micro Score:", f1_micro)


In [19]:
Y_pred = clf.predict(X_valid)

In [20]:
print('For both Level 1 and Level 2  concatenated:\n\tF1 micro (=accuracy): {}'.format(f1_01(Y_valid,Y_pred).round(3)))

For both Level 1 and Level 2  concatenated:
	F1 micro (=accuracy): 0.113


In [21]:
print('Just the Level 1:\n\tF1 micro (=accuracy): {}'.format(f1_0(Y_valid,Y_pred).round(3)))

Just the Level 1:
	F1 micro (=accuracy): 0.238


In [22]:
print('Just the Level 2:\n\tF1 micro (=accuracy): {}'.format(f1_1(Y_valid,Y_pred).round(3)))

Just the Level 2:
	F1 micro (=accuracy): 0.118
